In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.layers import Input, Dense, Activation,BatchNormalization, Flatten, Convolution2D, MaxPooling2D, Dropout
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
import h5py

In [2]:
path = r"C:\Users\GUITA\Desktop\Stuff\Projects\Plant CLEF\alexnet\dataset\seg_train\seg_train"
datagen = ImageDataGenerator(validation_split=0.2, rescale=1. / 255)
train = datagen.flow_from_directory(path, subset='training', target_size=(227,227), class_mode='categorical')
test = datagen.flow_from_directory(path, subset='validation', target_size=(227,227), class_mode='categorical')

Found 5060 images belonging to 135 classes.
Found 1199 images belonging to 135 classes.


In [16]:
classifier = Sequential()

# Convolution Step 1
classifier.add(Convolution2D(96, 11, strides = (4, 4), padding = 'valid', input_shape=(224, 224, 3), activation = 'relu'))

# Max Pooling Step 1
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
classifier.add(BatchNormalization())

# Convolution Step 2
classifier.add(Convolution2D(256, 11, strides = (1, 1), padding='valid', activation = 'relu'))

# Max Pooling Step 2
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding='valid'))
classifier.add(BatchNormalization())

# Convolution Step 3
classifier.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
classifier.add(BatchNormalization())

# Convolution Step 4
classifier.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
classifier.add(BatchNormalization())

# Convolution Step 5
classifier.add(Convolution2D(256, 3, strides=(1,1), padding='valid', activation = 'relu'))

# Max Pooling Step 3
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
classifier.add(BatchNormalization())

# Flattening Step
classifier.add(Flatten())

# Full Connection Step
classifier.add(Dense(units = 4096, activation = 'relu'))
#classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 4096, activation = 'relu'))
#classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 1000, activation = 'relu'))
#classifier.add(Dropout(0.2))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 38, activation = 'softmax'))


In [17]:
classifier.load_weights(r"C:\Users\GUITA\Desktop\Stuff\Projects\Plant CLEF\alexnet\AlexNetModel.hdf5")

In [18]:
classifier = Model(inputs=classifier.input, outputs=classifier.layers[-2].output)

In [19]:
classifier.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5_input (InputLayer)  [(None, 224, 224, 3)]    0         
                                                                 
 conv2d_5 (Conv2D)           (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 27, 27, 96)       0         
 2D)                                                             
                                                                 
 batch_normalization_8 (Batc  (None, 27, 27, 96)       384       
 hNormalization)                                                 
                                                                 
 conv2d_6 (Conv2D)           (None, 17, 17, 256)       2973952   
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 8, 8, 256)        0   

In [20]:
model=Sequential()

In [21]:
model.add(classifier)

In [22]:
model.add(Dense(units = 135, activation = 'softmax'))

In [23]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_1 (Functional)        (None, 1000)              28079752  
                                                                 
 dense_9 (Dense)             (None, 135)               135135    
                                                                 
Total params: 28,214,887
Trainable params: 28,193,751
Non-trainable params: 21,136
_________________________________________________________________


In [24]:
#opt = tf.optimizers.Adagrad(learning_rate=0.00495, epsilon=None, decay=0.0)
model.compile(optimizer = 'SGD' , loss = tf.keras.losses.KLDivergence() , metrics=['accuracy'])

In [ ]:
model.fit(train,epochs=20)

Epoch 1/20
159/159 [==============================] - 169s 1s/step - loss: 4.4769 - accuracy: 0.1079
Epoch 2/20
159/159 [==============================] - 134s 841ms/step - loss: 3.4815 - accuracy: 0.2051
Epoch 3/20
159/159 [==============================] - 136s 853ms/step - loss: 3.0271 - accuracy: 0.2731
Epoch 4/20
159/159 [==============================] - 138s 869ms/step - loss: 2.6921 - accuracy: 0.3265
Epoch 5/20
159/159 [==============================] - 159s 1s/step - loss: 2.3893 - accuracy: 0.3911
Epoch 6/20
159/159 [==============================] - 160s 1s/step - loss: 2.0659 - accuracy: 0.4623
Epoch 7/20
159/159 [==============================] - 164s 1s/step - loss: 1.7488 - accuracy: 0.5462
Epoch 8/20
159/159 [==============================] - 164s 1s/step - loss: 1.6768 - accuracy: 0.5561
Epoch 9/20
159/159 [==============================] - 139s 873ms/step - loss: 1.5033 - accuracy: 0.6030
Epoch 10/20
159/159 [==============================] - 140s 879ms/step - loss: 

In [14]:
preds=model.evaluate(test) 
print("Loss = " + str(preds[0])) 
print("Test Accuracy = " + str(preds[1]))

38/38 [==============================] - 11s 274ms/step - loss: 3.2849 - accuracy: 0.2777
Loss = 3.2848544120788574
Test Accuracy = 0.27773144841194153
